# First Attempt
Max Fierro 2/20/2022

### From Deterministic and Stochastic Predictions to Performing Lookups on an Adjacency Matrix

This is my first exercise working with both a jupyter notebook and the concept of a Markov chain. The idea is one I was briefly exposed to as a sidenote in one of my classes, CS61a at UC Berkeley in Fall '21. I think I remember one of my instructors, Pamela Fox, briefly going into a tangent of how you could sort of emulate what someone could say by making a 'dictionary' of words from a text, in such a way that every `key:value` pair is every `word:nextword`. It's pretty simple, and it has the somewhat cool property that obtaining `dict[word]` would (if computers were not so annoyingly deterministic) fetch some `nextword`, where the probability of choosing it depends on the amount of times it appears after `word` divided by the total number of appearances of `word` in the text.

In [1]:
import random
import time

### Initial implementation

This also being my first 'real' practical experience working with data, I thought I should be as verbose as possible and try to identify some of the challenges of working with data before trying to work with some libraries. While I know this is obviously not the most elegant way of doing this, the code is pretty easy to understand, and the methods allow for some good old `print` calls here and there if you want to see what is happening (TBH I also needed a tiny refresher with Python).

BTW, I do not endorse the language used in 'Adventures of Huckleberry Finn' by Mark Twain, in case you are either stochastically or deterministically delivered a no-no word.

   So! Here is what I came up with first:
1. A `PairLink` class defining `word:nextword` pairs.

In [11]:
class PairLink:
    
    def __init__(self, key, value, unwanted = []):
        self._unwanted = unwanted
        self._val = value
        self._key = key
    
    def clean(self, f):
        def alter(elt):
            if elt == None: return None
            for item in self._unwanted:
                if type(elt) == str:
                    elt = elt.replace(item, '')
            return f(elt) if f else elt
        self._val = alter(self._val)
        self._key = alter(self._key)
    
    def val(self):
        return self._val
    
    def key(self):
        return self._key
    
    def __repr__(self):
        return str(self._key) + ' -> ' + str(self._val)

2. A `MkvPool` class defining a collection of `PairLink`s.

In [12]:
class MkvPool:
    
    def __init__(self, pair_link_list):
        self._list = [l for l in pair_link_list]
    
    def clean_all(self, f):
        [l.clean(f) for l in self._list]
    
    def item_list(self):
        return [item for item in dic(self).keys()]
    
    def size(self):
        return len(self._list)
    
    def lst(self):
        return self._list
    
    def dic(self):
        return {l.key(): l.val() for l in self._list}
    
    def choose_with_key(self, key):
        return [l for l in self._list if l.key() == key]
    
    def __repr__(self):
        repr([self.lst()])
    
    def next_deterministic(self, first):
        k = {}
        for l in self.lst():
            if l.key() == first:
                if l.val() not in k.keys():
                    k[l.val()] = 1
                else:
                    k[l.val()] += 1
        return max(k, key = k.get)

3. The function `read_txt` which unloads `chars` characters from a .txt file into a list of clean word strings free of `unwanted` characters, and `word_PairLinks` which turns them into `PairLink`s. This is where our data, "Adventures of Huckleberry Finn," is processed. If you notice any redundancy, it is there so I can use `read_txt` later.

In [13]:
def read_words(file, unwanted, chars = None):
    doc = open(file)
    data = doc.read(chars).split()
    doc.close()
    words = []
    for word in data:
        for item in unwanted:
            word = word.replace(item, '')
        word = word.lower()
        words.append(word)
    return words

def word_PairLinks(words, unwanted):
    links = [PairLink(None, words[0], unwanted)]
    for i in range(len(words) - 2):
        links.append(PairLink(words[i], words[i + 1], unwanted))
    links.append(PairLink(words[-1], None, unwanted))
    return links

unwanted = ['"', '^', '*', '-', '_', '/', '[', ']', '<', '>', '~', ',']

words = read_words('data/huck_finn.txt', unwanted, 1000000)
pool = MkvPool(word_PairLinks(words, unwanted))

4. The funciton `make_chain` which creates a chain of words using some `choose_next_func` to choose the next word based on the current one.

In [14]:
def make_chain(length, choose_next_func, seed = None):
    counter, last = 0, seed
    chain = '' if not seed else seed
    while counter < length:
        nxt = choose_next_func(last)
        chain += ' ' + nxt if nxt != None else ''
        last = nxt
        counter += 1
    return chain

5. Some `choose_next_func`s, which deliver a following word stochastically and deterministically.

In [15]:
def choose_stochastically(last, bank = pool):
    return random.choice(bank.choose_with_key(last)).val()

def choose_deterministically(last, bank = pool):
    return bank.next_deterministic(last)

### Results (PairLink implementation)
While this served its purpose as a first exercise, it obviously constitutes a very poor model when it comes to prediction. You would need to get very lucky to get something coherent. It also has a lot of unnecessary stuff which I made for the sake of killing time and reviewing Python, which probably made it very unefficient.

In [16]:
"""
You can change the length and seed of the predictions. 

Notice how the output of the deterministic chain eventually stabilizes into
repetition. Exercise for the reader: Determine, on average, after how many 
words the deterministic chain exhibits cyclic behavior given some seed word W 
in some dataset D.

Now that I've left an exercise for the reader, I can die in peace.
"""

print('\nDeterministic chain:\n\n', make_chain(30, choose_deterministically, 'hello'))
start = time.time()
print('\nStochastic chain:\n\n', make_chain(30, choose_stochastically, 'hello'))
print('\nTime elapsed for stochastic chain:', time.time() - start)


Deterministic chain:

 hello jim said he was a little and the king he was a little and the king he was a little and the king he was a little and the king

Stochastic chain:

 hello jim got the new kind of them out only just as you needn't keep a rattling kick. but uncle and they'll soon he said they'd just to make certain. now

Time elapsed for stochastic chain: 0.4450669288635254


### Looking for improvement

To hopefully improve our performance, let's do a little bit of math. I'll define a matrix `M` with a column and row for each word, where $M_{i, j} = P(j | i)$. In other words, every element of $M$ will represent the probability that the word corresponding to its column follows the word corresponding to its row within the text we are analyzing. I read about this matrix representation somewhere a while ago, so while I am unable to point to a reference it should suffice to say that, unlike the last implementation, I did not come up with this by myself.

The objective of making this matrix will be to determine which word should go after another more quickly, allowing us to make quicker predictions at the cost of a slower initial matrix computation. Again, I'll keep all libraries out of this for now for the sake of my learning, and the only thing I'll be using from the previous section is our `words` list, returned by `read_words`.

In [17]:
def create_matrix(all_words):
    
    def init_matrix(uniques):
        return [[0] * len(uniques) for i in range(len(uniques))]
    
    unique_words = list(set(all_words))
    uw_indices = {word:unique_words.index(word) for word in unique_words}
    M = init_matrix(unique_words)
    successor_count = {}
    
    for i in range(len(all_words) - 1):
        curr = uw_indices[all_words[i]]
        nxt = uw_indices[all_words[i + 1]] 
        M[curr][nxt] += 1
    
    for i in range(len(M)):
        successor_count[i] = sum(M[i])
        M[i] = [p / successor_count[i] for p in M[i]]
    
    return M, unique_words, uw_indices

M, Uwords, Uindices = create_matrix(words)

This is set up such that $M_{i,j}$ = P(`Uword[j]` | `Uword[i]`), which I feel could be made simpler, but any improvements elude me at the moment. Anyway, we now have a matrix correlating each word and their possible successors probabilistically.

Now, to create a `choose_next_func` which takes advantage of my hard work:

In [18]:
def choose_stochastic_matrix(last, m = M, uw = Uwords, ui = Uindices):
    possible_words = m[ui[last]]
    tmp, i = [], 0
    for word in possible_words:
        tmp += [[word, i]]
        i += 1
    result = random.choices(tmp, possible_words, k = 1)[0][1]
    return uw[result]

### Results (Matrix implementation)

Again, I will not pretend that this model provides an accurate prediction of what Mark Twain would say.

In [19]:
start = time.time()
print('\nStochastic matrix prediction:\n\n', make_chain(100, choose_stochastic_matrix, 'hello'))
print('\nTime elapsed for matrix chain:', time.time() - start)


Stochastic matrix prediction:

 hello jim says: miss them. gimme no more than what she was going to slide down the expense of use in town if i was worth of a town so on. i took a good place. she started her lights twinkling where my craw; but somehow it wouldn't be mighty smart. yes he done some little and shook and ready to the originator of men jumped in the oldest man to meand then pretty silly to get there; you 'bout? i must up very thick which was saying is i was all up my ears; and live! and your names. the

Time elapsed for matrix chain: 8.515828847885132


Okay, I have to admit I was not expecting that. My theory was that creating a matrix with all the probability information would save a lot of computation, and therefore a lot of time. But apparently not, and it instead consistently triples the time required to create a chain (at least for smaller output lengths -- I do not have the time to experiment with larger chains).

Having completed that, I feel like I have gotten my foot in the door to working with Markov chains and with processing data, so I will call it a day. If you know why my hypothesis was incorrect or how to fix my implementation, please email me. I would love to know.